## Economic Dispatch (ED)
_Power System Optimization (with pyomo and linopy)_

System operators must select the most cost effective combination of generators, each with distinct cost structures and engineering limitations. ED is a key operational model for the optimization of the variable (short-run) production costs of a fleet of generators, subject to various technical constraints, to meet the total electricity demand. 

- **Characteristics**:
    - Objective: Minimize short-run costs of different producing generators while meeting the total system damand, considering relevant constraints.
    - Constraints: Power balance (total generation = total demand), generation limits (minimum and maximum capacity), and sometimes ramp rates for generators.
    - Assumptions: Assumes the system can adjust generation output in real-time to minimize cost while meeting demand, without considering transmission constraints.Physics of electricity flows and constraints related to turning on or "committing" large thermal generators aswell as network representation are neglected  

- **Advantages**: 
    - Simple and efficient for real-time dispatch decisions.
- **Disadvantages**:
    - Ignores power system constraints like voltage, line flow, and losses.
    - Cannot handle reactive power optimization or voltage stability concerns.

- **Field of Application**:
    - Real-time and short-term dispatch in power markets.
    - Used in situations where the main concern is balancing generation and demand economically without consideration of network constraints.

>[!NOTE]  
>fix costs will not change and would be constant in the objective function -> optimal decision variables would not change by adjusting this constant, therefore, they can safely be ignored for the purpose of finding optimal dispatch, but will have to be considered to calculate producer profits.

### Different complexity levels:  
1. ED: [Single-time period, simple generator constraints](#1-single-time-period-simple-generator-constraints)
2. DED: [Multiple-time period, simple generator constraints](#2-multiple-time-period-simple-generator-constraints)
3. DED with ramp rates: [Multiple-time period, complex generator constraints with time coupling (_including ramp up and ramp down_)](#3-multiple-time-period-complex-generator-constraints-with-time-coupling)
4. DED with ramp rates and battery: [Multiple-time period, time coupling + battery](#4-multiple-time-period-time-coupling--battery)  

... next levels: + power transmission (see Optimal Power Flow)

### 1. Single-time period, simple generator constraints

**Problem Formulation**:  
Determine the power generation by each generator, minimising the power generation cost while meeting the given static system demand and considering (only) the variable OM costs for each generator.

$$G = \text{set of generators}$$

Objective Function:  
$$min \sum_{g \in G} 
VarCost_g \times x_g$$

subject to:
$$\begin{aligned}
    \sum_g x_g & = Demand \\
    x_g &\le P^{max}_g  & \quad  \forall g \in G \\
    x_g &\ge P^{min}_g   & \quad \forall g \in G \\
\end{aligned}$$

Decision Variable:  
$$x_g: \text{generation in MW, produced by each generator g}$$

Parameters:

- $P^{min}_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $P^{max}_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand$ in MW
- Variable Costs:

$$
VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g
$$
with OM: operating and maintenance.  
For renewables (variable generators) it reduces to $VarCost_g = VarOM_g$

In [ ]:
import pandas as pd
df_generators=pd.read_csv('data/generators.csv', sep=',')
df_generators.fillna(0, inplace=True)

df_variables = df_generators.loc[df_generators.is_variable==1]
df_non_variables = df_generators.loc[df_generators.is_variable!=1]

df_generators

In [ ]:
df_variables

#### Pyomo Solution
[_Single-time period, simple generator constraints_](#1-single-time-period-simple-generator-constraints)

In [ ]:
import pyomo.environ as pyo

n_generators = len(df_generators)
const_total_demand = 400

model = pyo.ConcreteModel()

# Sets
model.G = pyo.RangeSet(n_generators)
model.G_var = pyo.Set(initialize=df_variables.index+1)
model.G_non_var = pyo.Set(initialize=df_non_variables.index+1)

# Decision Variable
def bounds_rule(model, g):
    return (df_generators.Pmin[g-1], df_generators.Pmax[g-1])
model.x = pyo.Var(
    model.G, 
    within=pyo.NonNegativeReals, 
    bounds=bounds_rule,
)

# Params are defined for the whole generator set model.G, because, as HeatRate and FuelCost are set to 0, 
# VarCost is automatically reduced to VarOM for variable generators.
model.VarOM = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.VarOM)))
model.HeatRate = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.HeatRate)))
model.FuelCost = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.FuelCost)))
model.CF = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.CF)))
model.Pmax = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.Pmax)))

# Objective 
model.total_costs=pyo.Objective(
    expr=sum(
        model.x[g] * (model.VarOM[g] + model.HeatRate[g] * model.FuelCost[g]) 
        for g in model.G
    ),
    sense=pyo.minimize
)

# Constraints
def demand_rule(model):
    return sum(model.x[g] for g in model.G) == const_total_demand
model.demand_constraint = pyo.Constraint(rule=demand_rule)

# capacity_constraint_non_var not necessary due to bounds rule

def capacity_rule_var(model, g):
    return model.x[g] == model.CF[g] * model.Pmax[g]
model.capacity_constraint_var = pyo.Constraint(model.G_var, rule=capacity_rule_var)

optimizer = pyo.SolverFactory("appsi_highs")
result = optimizer.solve(model)

result.write()
print(f"total costs: {pyo.value(model.total_costs)}")
# model.x.display()
pd.Series(model.x.extract_values()).plot(grid=True, kind="bar", title="Pyomo Solution", xlabel="#-generator", ylabel="MW")


#### Linopy Solution
[_Single-time period, simple generator constraints_](#1-single-time-period-simple-generator-constraints)

In [ ]:
import linopy 

model = linopy.Model()

const_total_demand = 400

# Decision Variable
x = model.add_variables(
    lower=df_generators.Pmin, 
    upper=df_generators.Pmax,
    coords=[df_generators.index],
    name="generators"
)

# no direct need to define Parameters and Sets with linopy

# Constraints
demand_constraint = model.add_constraints(x.sum() == const_total_demand)

def var_capacity_rule(model, i):
    return x[i] == df_variables.CF[i] * df_variables.Pmax[i]
capacity_constraint_var = model.add_constraints(var_capacity_rule ,coords=[df_variables.index])

def non_var_capacity_rule(model, i):
    return x[i] <= df_non_variables.Pmax[i]
capacity_constraint_non_var = model.add_constraints(non_var_capacity_rule, coords=[df_non_variables.index])

# Objective
obj = (x * (df_generators.VarOM + df_generators.HeatRate * df_generators.FuelCost)).sum()
model.add_objective(obj, overwrite=True, sense='min')

model.solve()


In [ ]:
solution = model.solution.to_dataframe()
solution.plot(grid=True, kind="bar", title="Linopy Solution", xlabel="#-generator", ylabel="MW")

### 2. Multiple-time period, simple generator constraints
  
**Problem Formulation**:  
Determine the power generation by each generator, minimising the power generation cost while meeting the given dynamic system demand and considering (only) the variable OM costs for each generator.  
The time dependency of the demand and the generation is to be taken into account.

-> an additional time index t is introduced:
      
      
Objective Function:
$$ \min \sum_{g \in G, t \in T} VarCost_g \times x_{g,t} $$
subject to:
$$\begin{align}
    \sum_{g} x_{g,t} = Demand_t \quad \forall  t \in T \\
    x_{g,t} \leq P^{max}_{g,t} \quad \forall g \in G , t \in T \\
    x_{g,t} \geq P^{min}_{g,t} \quad \forall g \in G , t \in T \\
\end{align}$$
    
- for conventional resources Pmin and Pmax are constant over time
- for variable (renewable) generators, $Pmax_{g,t}$ varies with time (i.e., based on solar irradiation or wind speeds). $Pmin_{g,t}$ for wind and solar resources is usually 0.
- for hydropower resources minimum streamflow constraints can produce a time-variant parameter for $Pmin_{g,t}$.
- each time period is balanced separately without regard to what is happening before.

### Pyomo Solution
[_Multiple-time period, simple generator constraints_](#2-multiple-time-period-simple-generator-constraints)  

In [ ]:
import requests
import pandas as pd

url = "https://api.energy-charts.info/total_power"

end = pd.Timestamp.today() -pd.Timedelta(days=1)
start = end - pd.Timedelta(days=2)

params = {
    "country": "de" ,
    "start": round(start.timestamp()), 
    "end": round(end.timestamp()),
}

response = requests.get(url=url, params=params)

response.text

In [2]:

data = response.json()
index = data.pop(list(data.keys())[0])  # rely on dict order

df_dyn_generation_energy_charts = pd.DataFrame(
    {item['name']:item['data'] for item in data['production_types']},
    index=pd.DatetimeIndex(pd.to_datetime(index, unit='s'))
)

GW_to_MW = 1000
installed_capacity = pd.read_csv('data/energy-charts_Installierte_Netto-Leistung_de_2024.csv', index_col='Jahr').T.astype({2024:'float'})
installed_capacity.drop(installed_capacity.columns[0],axis=1, inplace=True)
installed_capacity = installed_capacity *  GW_to_MW 

installed_total_capacity = installed_capacity[2024].sum()


In [ ]:
import pyomo.environ as pyo

n_generators = len(df_generators.index)
df_dyn_total_demand = df_dyn_generation_energy_charts["Load (incl. self-consumption)"] / installed_total_capacity * df_generators.Pmax.sum()  # simulate only partial network demand

model = pyo.ConcreteModel()

# Sets
model.G = pyo.RangeSet(n_generators)
model.G_var = pyo.Set(initialize=df_variables.index+1)
model.G_non_var = pyo.Set(initialize=df_non_variables.index+1)
model.T = pyo.RangeSet(len(df_dyn_generation_energy_charts))


# Decision Variable
def bounds_rule(model, g, t):
    return (df_generators.Pmin[g-1], df_generators.Pmax[g-1])
model.x = pyo.Var(
    model.G, 
    model.T,
    within=pyo.NonNegativeReals, 
    bounds=bounds_rule,
)

# Params 

init_CF = {(g, t): 
    df_dyn_generation_energy_charts[df_generators.loc[g-1].Resource].iloc[t-1]/ installed_capacity.loc[df_generators.loc[g-1].Resource, 2024]
    for g in model.G_var 
    for t in model.T
} # simulate time dependent cf factors with real generation/ real installed capacity

model.VarOM = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.VarOM)))
model.HeatRate = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.HeatRate)))
model.FuelCost = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.FuelCost)))
model.CF = pyo.Param(model.G, model.T, initialize=init_CF)
model.Pmax = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.Pmax)))
model.Demand = pyo.Param(model.T, initialize=dict(zip(model.T, df_dyn_total_demand)))  


# Objective 
model.total_costs=pyo.Objective(
    expr=sum(
        model.x[g,t] * (model.VarOM[g] + model.HeatRate[g] * model.FuelCost[g]) 
        for g in model.G for t in model.T
    ),
    sense=pyo.minimize
)

# Constraints
def demand_rule(model, t):
    return sum(model.x[g,t] for g in model.G) == model.Demand[t]
model.demand_constraint = pyo.Constraint(model.T, rule=demand_rule)

# capacity_constraint_non_var not necessary due to bounds rule

def capacity_rule_var(model, g, t):
    return model.x[g, t] == model.CF[g, t] * model.Pmax[g]
model.capacity_constraint_var = pyo.Constraint(model.G_var, model.T, rule=capacity_rule_var)

optimizer = pyo.SolverFactory("appsi_highs")
result = optimizer.solve(model)

result.write()
print(f"total costs: {pyo.value(model.total_costs)}")
# model.x.display()


In [ ]:
# Visualize optimized power generation 

import matplotlib.pyplot as plt 

data = model.x.extract_values()
index = df_dyn_generation_energy_charts.index

df_data = pd.Series(data).unstack(level=0)
df_data.index = index
df_data.columns=[f"{df_generators.loc[generator-1].Resource}" for generator in df_data.columns]

for generator in df_data.columns:
    plt.plot(df_data[generator], label=generator)
plt.plot(df_dyn_total_demand, label='Demand')

plt.legend(title='Generators')
plt.tick_params(axis='x', rotation=55)
plt.show()


In [ ]:
# Visualize stacked optimized power generation
#  
ax = df_data[[ 'Biomass', 'Gas_Turbine', 'Wind offshore', 'Wind onshore', 'Solar']].plot(kind='area', stacked=True, figsize=(10, 6), alpha = 0.7)
df_dyn_total_demand.plot(ax=ax, label='Demand', linestyle='--',color='red', linewidth=2)

plt.title('Stacked Power Generation with Demand Over Time')
plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.grid(True)
plt.legend(title='Generators and Demand',loc='lower right')

## 3. Multiple-time period, complex generator constraints with time coupling
   
**Problem Formulation**:  
Determine the power generation by each generator, minimizing the power generation cost while meeting the given dynamic system demand and considering (only) the variable OM costs for each generator.  
The time dependency of the demand and the generation, aswell time coupling, is to be taken into account.
  

-> Time coupling is introduced by ramp rates, which limit the change in the generators output from one time period to the next one:
  
Objective Function:
$$\min \sum_{g \in G, t \in T} VarCost_g \times x_{g,t} $$
subject to:
$$\begin{align}
    \sum_{g} x_{g,t} = Demand_t & \quad \forall t \in T \\
    x_{g,t} \leq Pmax_{g,t} &  \quad \forall g \in G , t \in T \\
    x_{g,t} \geq Pmin_{g,t} &  \quad \forall g \in G , t \in T \\
    x_{g,t+1} - x_{g,t} \leq RampUp_{g} & \quad \forall g \in G , t = 1..T-1 \\
    x_{g,t} - x_{g,t+1} \leq RampDown_{g} &  \quad \forall g \in G , t = 1..T-1
\end{align}$$

In [ ]:
import pyomo.environ as pyo

n_generators = len(df_generators.index)
df_dyn_total_demand = df_dyn_generation_energy_charts["Load (incl. self-consumption)"] / installed_total_capacity * df_generators.Pmax.sum()  # simulate only partial network demand

model = pyo.ConcreteModel()

# Sets
model.G = pyo.RangeSet(n_generators)
model.G_var = pyo.Set(initialize=df_variables.index+1)
model.G_non_var = pyo.Set(initialize=df_non_variables.index+1)
model.T = pyo.RangeSet(len(df_dyn_generation_energy_charts))


# Decision Variable
def bounds_rule(model, g, t):
    return (df_generators.Pmin[g-1], df_generators.Pmax[g-1])
model.x = pyo.Var(
    model.G, 
    model.T,
    within=pyo.NonNegativeReals, 
    bounds=bounds_rule,
)

# Params 

init_CF = {(g, t): 
    df_dyn_generation_energy_charts[df_generators.loc[g-1].Resource].iloc[t-1]/ installed_capacity.loc[df_generators.loc[g-1].Resource, 2024]
    for g in model.G_var 
    for t in model.T
} # simulate time dependent cf factors with real generation/ real installed capacity

model.VarOM = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.VarOM)))
model.HeatRate = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.HeatRate)))
model.FuelCost = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.FuelCost)))
model.CF = pyo.Param(model.G, model.T, initialize=init_CF)
model.Pmax = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.Pmax)))
model.Demand = pyo.Param(model.T, initialize=dict(zip(model.T, df_dyn_total_demand)))
model.ramp_up_percentage = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.RampUp)))
model.ramp_dn_percentage = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.RampDn)))


# Objective 
model.total_costs=pyo.Objective(
    expr=sum(
        model.x[g,t] * (model.VarOM[g] + model.HeatRate[g] * model.FuelCost[g]) 
        for g in model.G for t in model.T
    ),
    sense=pyo.minimize
)

# Constraints
def demand_rule(model, t):
    return sum(model.x[g,t] for g in model.G) == model.Demand[t]
model.demand_constraint = pyo.Constraint(model.T, rule=demand_rule)

# capacity_constraint_non_var not necessary due to bounds rule

def capacity_rule_var(model, g, t):
    return model.x[g, t] == model.CF[g, t] * model.Pmax[g]
model.capacity_constraint_var = pyo.Constraint(model.G_var, model.T, rule=capacity_rule_var)

def ramp_up_rule(model, g, t):
    if t < len(model.T):
        return model.x[g,t+1] - model.x[g, t] <= model.Pmax[g] * model.ramp_up_percentage[g]
    else:
        return pyo.Constraint.Skip
model.ramp_up_constraint=pyo.Constraint(model.G, model.T, rule=ramp_up_rule)

def ramp_down_rule(model, g, t):
    if t < len(model.T):
        return model.x[g,t] - model.x[g, t+1] <= model.Pmax[g] * model.ramp_dn_percentage[g]
    else:
        return pyo.Constraint.Skip
model.ramp_dn_constraint=pyo.Constraint(model.G, model.T, rule=ramp_down_rule)

optimizer = pyo.SolverFactory("appsi_highs")
result = optimizer.solve(model)

result.write()
print(f"total costs: {pyo.value(model.total_costs)}")
# model.x.display()


In [ ]:
# Visualize optimized power generation 

import matplotlib.pyplot as plt 

data = model.x.extract_values()
index = df_dyn_generation_energy_charts.index

df_data = pd.Series(data).unstack(level=0)
df_data.index = index
df_data.columns=[f"{df_generators.loc[generator-1].Resource}" for generator in df_data.columns]

for generator in df_data.columns:
    plt.plot(df_data[generator], label=generator)
plt.plot(df_dyn_total_demand, label='Demand')

plt.legend(title='Generators')
plt.tick_params(axis='x', rotation=55)
plt.show()


## 4. Multiple-time period, time coupling + battery
   
**Problem Formulation**:  
Determine the power generation by each generator, minimizing the power generation cost while meeting the given dynamic system demand and considering (only) the variable OM costs for each generator.  
The time dependency of the demand and the generation, aswell time coupling, is to be taken into account.
  

-> Time coupling is introduced by ramp rates, which limit the change in the generators output from one time period to the next one:
  
Objective Function:
$$\min \sum_{g \in G, t \in T} VarCost_g \times x_{g,t} $$
subject to:
$$\begin{align}
    \sum_{g} x_{g,t} + P_{dch, t} = Demand_t + P_{ch, t}& \quad \forall g \in G, t \in T \\
    Pmin_{g} \leq x_{g,t} \leq Pmax_{g} &  \quad \forall g \in G , t \in T \\
    SoC_t = SoC_{t-1} + (\mu P_{ch, t} - \frac{P_{dch, t}} {\mu})\Delta t & \quad \forall t \in T\\
    SoC^{min} \leq SoC_t \leq SoC^{max} &  \quad \forall t \in T \\
    0 \leq P_{ch,t} \leq P^{max}_{ch} &  \quad \forall t \in T \\
    0 \leq P_{dch,t} \leq P^{max}_{dch} &  \quad \forall t \in T \\
    x_{g,t+1} - x_{g,t} \leq RampUp_{g} & \quad \forall g \in G , t = 1..T-1 \\
    x_{g,t} - x_{g,t+1} \leq RampDown_{g} &  \quad \forall g \in G , t = 1..T-1
\end{align}$$

In [ ]:
import pandas as pd
df_generators=pd.read_csv('data/generators.csv', sep=',')
df_generators.fillna(0, inplace=True)

df_variables = df_generators.loc[df_generators.is_variable==1]
df_non_variables = df_generators.loc[df_generators.is_variable!=1]

df_generators

df_storage = pd.read_csv('data/storage.csv', index_col='Resource')
df_storage

In [ ]:
import pyomo.environ as pyo

n_generators = len(df_generators.index)
df_dyn_total_demand = df_dyn_generation_energy_charts["Load (incl. self-consumption)"] / installed_total_capacity * df_generators.Pmax.sum() /1.1  # simulate only partial network demand


# battery parameters
battery_Emax = df_storage.loc['Battery', 'Emax'] #MWh
battery_initial_SoC = 0.5 * battery_Emax #MWh
battery_charge_efficiency = 0.95
battery_discharge_efficiency = 0.9
battery_max_charge = 100 #MW
battery_max_discharge = 100 #MW


model = pyo.ConcreteModel()

# Sets
model.G = pyo.RangeSet(n_generators)
model.G_var = pyo.Set(initialize=df_variables.index+1)
model.G_non_var = pyo.Set(initialize=df_non_variables.index+1)
model.T = pyo.RangeSet(len(df_dyn_generation_energy_charts))


# Decision Variable
def bounds_rule(model, g, t):
    return (df_generators.Pmin[g-1], df_generators.Pmax[g-1])
model.x = pyo.Var(
    model.G, 
    model.T,
    within=pyo.NonNegativeReals, 
    bounds=bounds_rule,
)


# Variables
model.SoC = pyo.Var(model.T, within=pyo.NonNegativeReals, initialize=battery_initial_SoC, bounds=(0, battery_Emax))
model.Battery_charge = pyo.Var(model.T, within=pyo.NonNegativeReals, initialize=0, bounds=(0, battery_max_charge))
model.Battery_discharge = pyo.Var(model.T, within=pyo.NonNegativeReals, initialize=0, bounds=(0, battery_max_discharge))
model.u = pyo.Var(model.T, within=pyo.Binary) # charge / discharge switch

# Params 

init_CF = {(g, t): 
    df_dyn_generation_energy_charts[df_generators.loc[g-1].Resource].iloc[t-1]/ installed_capacity.loc[df_generators.loc[g-1].Resource, 2024]
    for g in model.G_var 
    for t in model.T
} # simulate time dependent cf factors with real generation/ real installed capacity

model.VarOM = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.VarOM)))
model.HeatRate = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.HeatRate)))
model.FuelCost = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.FuelCost)))
model.CF = pyo.Param(model.G, model.T, initialize=init_CF)
model.Pmax = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.Pmax)))
model.Demand = pyo.Param(model.T, initialize=dict(zip(model.T, df_dyn_total_demand)))

model.ramp_up_percentage = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.RampUp)))
model.ramp_dn_percentage = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.RampDn)))

model.battery_initial_SoC=pyo.Param(initialize=battery_initial_SoC)
model.battery_charge_efficiency = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_charge_efficiency)
model.battery_discharge_efficiency = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_discharge_efficiency)
model.battery_max_charge = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_max_charge)
model.battery_max_discharge = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_max_discharge)


# Objective 
model.total_costs=pyo.Objective(
    expr=sum(
            model.x[g,t] * (model.VarOM[g] + model.HeatRate[g] * model.FuelCost[g]) 
            for g in model.G 
            for t in model.T
        ),
    sense=pyo.minimize
)

# Constraints
def demand_rule(model, t):
    return sum(model.x[g,t] for g in model.G) + model.Battery_discharge[t] == model.Demand[t] + model.Battery_charge[t]
model.demand_constraint = pyo.Constraint(model.T, rule=demand_rule)

# capacity_constraint_non_var not necessary due to bounds rule

def capacity_rule_var(model, g, t):
    return model.x[g, t] == model.CF[g, t] * model.Pmax[g]
model.capacity_constraint_var = pyo.Constraint(model.G_var, model.T, rule=capacity_rule_var)

def ramp_up_rule(model, g, t):
    if t < len(model.T):
        return model.x[g,t+1] - model.x[g, t] <= model.Pmax[g] * model.ramp_up_percentage[g]
    else:
        return pyo.Constraint.Skip
model.ramp_up_constraint = pyo.Constraint(model.G, model.T, rule=ramp_up_rule)

def ramp_down_rule(model, g, t):
    if t < len(model.T):
        return model.x[g,t] - model.x[g, t+1] <= model.Pmax[g] * model.ramp_dn_percentage[g]
    else:
        return pyo.Constraint.Skip
model.ramp_dn_constraint=pyo.Constraint(model.G, model.T, rule=ramp_down_rule)

def soc_rule(model, t):
    if t == 1:
        #return pyo.Constraint.Skip
        return model.SoC[t] == model.battery_initial_SoC + model.battery_charge_efficiency * model.Battery_charge[t] - model.Battery_discharge[t] / model.battery_discharge_efficiency
    else:
        return model.SoC[t] == model.SoC[t-1] + model.battery_charge_efficiency * model.Battery_charge[t] - model.Battery_discharge[t] / model.battery_discharge_efficiency
model.SoC_dynamics = pyo.Constraint(model.T, rule=soc_rule)

#Binary Constraint model.u[t]: Prevent simultaneous charging and discharging
def charge_rule(model, t):
    return model.Battery_charge[t] <= model.battery_max_charge * model.u[t]
model.battery_charge_limit = pyo.Constraint(model.T, rule=charge_rule)

def discharge_rule(model, t):
    return model.Battery_discharge[t] <= model.battery_max_discharge  * (1 - model.u[t])
model.battery_discharge_limit = pyo.Constraint(model.T, rule=discharge_rule)



optimizer = pyo.SolverFactory("appsi_highs")
result = optimizer.solve(model)

result.write()
print(f"total costs: {pyo.value(model.total_costs)}")
# model.x.display()


In [13]:
# Visualize optimized power generation 

import matplotlib.pyplot as plt 

data = model.x.extract_values()
index = df_dyn_generation_energy_charts.index

df_data = pd.Series(data).unstack(level=0)
df_data.index = index
df_data.columns=[f"{df_generators.loc[generator-1].Resource}" for generator in df_data.columns]
df_data['Battery_out'] = [abs(pyo.value(model.Battery_discharge[t])) for t in model.T]
df_data['Battery_in'] = [-1*abs(pyo.value(model.Battery_charge[t])) for t in model.T]


In [ ]:

# Visualize stacked optimized power generation
#  

ax = df_data[['Battery_in', 'Battery_out', 'Biomass', 'Gas_Turbine', 'Wind offshore', 'Wind onshore', 'Solar']].plot(kind='area', stacked=True, figsize=(10, 6), alpha = 0.7)
df_dyn_total_demand.plot(ax=ax, label='Demand', linestyle='--',color='red', linewidth=2)
df_SOC=pd.DataFrame([pyo.value(model.SoC[t]) for t in model.T], index=index, columns=['SoC'])
df_SOC.plot(ax=ax, label='SoC',color='black')

plt.title('Stacked Power Generation with Demand Over Time')
plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.grid(True)
plt.legend(title='Generators and Demand',loc='lower right')

In [ ]:

X=[t for t in model.T]
XL=[str(t) for t in model.T]

SOC=[pyo.value(model.SoC[t]) for t in model.T]
Charge=[pyo.value(model.Battery_charge[t]) for t in model.T]
discharge=[pyo.value(model.Battery_discharge[t]) for t in model.T]

plt.plot(X,SOC,label='SOC')
plt.plot(X,Charge,label='Charge')
plt.plot(X,discharge,label='DisCharge')

plt.grid( which='major', color='black', linestyle='-')
plt.grid( which='minor', color='grey', linestyle='--')
plt.minorticks_on()
plt.legend()
plt.show()